# Instant Gratification!

Let's build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, be sure to have followed the instructions in the "README" file, including creating your API key with OpenAI and adding it to the `.env` file.

In [4]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

Troubleshooting if you have problems:

1. OpenAI takes a few minutes to register after you set up an account. If you receive an error about being over quota, try waiting a few minutes and try again.
2. As a fallback, replace the line `openai = OpenAI()` with `openai = OpenAI(api_key="your-key-here")` - while it's not recommended to hard code tokens in Jupyter lab, because then you can't share your lab with others, it's a workaround for now
3. Contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point.

In [6]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

In [8]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
# Let's try one out

ed = Website("https://www.blender.org/features/simulation/m")
print(ed.title)
print(ed.text)

Blender 4.2 Manual
Contents
Menu
Expand
Light mode
Dark mode
Auto light/dark mode
Hide navigation sidebar
Hide table of contents sidebar
Toggle site navigation sidebar
Blender 4.2 Manual
Toggle Light / Dark / Auto color theme
Toggle table of contents sidebar
Blender 4.2 Manual
Getting Started
About Blender
Installing Blender
Configuring Blender
Help System
Sections
User Interface
Editors
Toggle navigation of Editors
3D Viewport
Toggle navigation of 3D Viewport
Introduction
Startup Scene
Object Modes
Navigating
Toggle navigation of Navigating
Introduction
Navigation
Fly/Walk Navigation
Aligning
Perspective/Orthographic
Local View
Camera View
Viewpoint
View Regions
Contextual Views
3D Cursor
Selecting
Controls
Toggle navigation of Controls
Transform Orientation
Pivot Point
Toggle navigation of Pivot Point
Bounding Box Center
3D Cursor
Individual Origins
Median Point
Active Element
Snapping
Proportional Editing
Display
Toggle navigation of Display
Object Type Visibility
Viewport Gizmos
Vi

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [12]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [16]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [18]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
summarize("https://www.blender.org/features/simulation/")

"# Blender Simulation Overview\n\nThe **Blender Simulation** page highlights Blender's capabilities in creating various simulations essential for 3D production. It focuses on providing tools for users who need realistic effects for projects, emphasizing:\n\n- **Smoke & Fire**: Create dynamic smoke and fire effects that interact with scenes.\n- **Fluids**: Simulate realistic water and other fluid dynamics.\n- **Hair**: Design lifelike hair that responds to environmental factors and collisions.\n- **Cloth**: Achieve realistic cloth behavior for garments and other materials.\n- **Rigid Bodies**: Enable destructible and collidable objects.\n- **Particles**: Generate effects like rain, sparks, and shrapnel.\n\nThe page positions Blender as a versatile tool for artists and developers looking for high-quality simulation results in their projects."

In [22]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [24]:
display_summary("https://docs.blender.org/manual/en/latest/physics/simulation_nodes.html")

# Simulation Nodes - Blender 4.2 Manual Summary

The **Simulation Nodes** section of the Blender 4.2 Manual provides insights into using geometry nodes to create custom physics simulations. It explains how **Simulation Zones** enable the results of one frame to affect the next, leading to complex outcomes from simple rules over time. 

## Key Features:
- **Baking Simulations**: Users can bake simulations to retain cache for playback, which is indicated by a yellow line in the timeline editor. This allows for efficiency when preparing animations for rendering.
- **Cache Management**: Caching solutions can be deleted if only the current frame matters, aiding in memory conservation.

## Examples:
The manual provides examples showcasing the combination of simulation nodes with the **Index of Nearest** for sphere-based simulations.

**Note**: It is important that the blend file is saved before baking, as this determines where the baked data will be stored.

Last updated on 2024-09-24.

In [26]:
display_summary("https://docs.blender.org/manual/en/latest/physics/baking.html")

# Summary of "Baking Physics Simulations - Blender 4.2 Manual"

The "Baking Physics Simulations" section of the Blender 4.2 Manual provides a comprehensive overview of the process for caching and baking physics simulations within Blender. It emphasizes that baking allows users to store the results of simulations in memory, which can significantly speed up rendering and ensure consistent results.

## Key Points:
- **Baking Definition**: Baking involves storing simulation results to protect the cache, preventing changes to the simulation settings until the baked data is cleared.
- **Two Cache Types**: Users can manage multiple caches simultaneously, with options to name and store them externally.
- **Disk Cache**: Cached simulations can be saved both inside the Blender file and externally, with options for compression to save space.
- **Simulation Control**: The section details controls for starting and ending frames, cache steps, and how to initiate baking processes. Options are available for partial baking and managing all simulations within the scene simultaneously.

This section aids users in optimizing their physics simulations for better performance and reliability in their designs. 

## Additional Notes:
- The baking operation makes the Blender interface temporarily unresponsive, displaying progress as a number.
- Users are reminded to set the End frame proactively for simulations longer than the default frame.

No specific news or announcements were mentioned in this portion of the website.

In [28]:
display_summary("https://docs.blender.org/manual/en/latest/physics/collision.html")

# Collision - Blender 4.2 Manual

The **Collision** section of the Blender 4.2 Manual provides detailed information about how various simulation objects interact with one another during physics simulations. This includes the behavior of particles, soft bodies, and cloth when they collide with mesh objects. Key points include:

- **Collision Mechanics**: Describes how soft bodies, cloth, particles, and hair interact with collision objects, emphasizing the importance of recalculating particle systems when collision settings are changed.
- **Collision Panel Options**:
  - **Field Absorption**: Controls how much of the effector's force is absorbed by colliders.
  - **Particle Properties**: Parameters such as permeability, stickiness, damping, and friction that govern particle interaction with colliders.
  - **Soft Body & Cloth Considerations**: Details on how the shape of the collider affects the simulation and the impact of the Collision Modifier's placement in the mesh topology.
- **Visual Examples**: Illustrations on how particles are deflected by mesh objects.
- **Practical Hints**: Tips for ensuring successful collisions, such as normal facing and applying scales correctly.

This section is crucial for users involved in animation and simulation in Blender, aiding them in setting up realistic interactions between different physical objects. 

*For further guidance, refer to sections on Baking Physics Simulations and Wind.* 

## Last Updated
- **Date**: 2024-09-24  
- **License**: CC-BY-SA 4.0 Int. License

---

This summary excludes any navigation-related content and focuses on the essential information provided in the Collision section of the Blender manual.

In [30]:
display_summary("https://docs.blender.org/manual/en/latest/physics/forces/force_fields/types/wind.html")

# Wind - Blender 4.2 Manual Summary

This webpage provides detailed documentation for the **Wind** force field feature in the Blender 4.2 manual. The Wind force field generates a constant force directed along the local Z-axis of the force object. The strength of the wind force is represented visually through the spacing of circles displayed in the interface.

## Key Features:
- **Functionality**: The Wind force field creates a directional force affecting objects within its range.
- **Visualization**: The strength of the wind is depicted by the spacing of visual indicators (circles) around the wind source.

This section is part of the broader Blender 4.2 manual, which encompasses a wide range of topics related to Blender's features, tools, and functionalities.

## Additional Notes:
- The document is part of Blender's official documentation, aimed at assisting users in understanding and utilizing its complex features effectively.
- The last update to this page was noted as September 24, 2024, indicating the information is regularly maintained. 

The manual includes many segments covering various aspects of Blender, from basic navigation and installation to advanced features and scripting, enhancing the user's experience in 3D modeling and animation.